In [1]:
from struct import unpack
from pathlib import Path
from enum import Enum

import json
import xmltodict
import numpy as np

import matplotlib.pyplot as plt

from astropy.time import Time
import astropy.units as u
from astropy.table import Table

from timdimm_tng.scheduler import Sequence, Observation, Schedule

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

In [8]:
s = Sequence(template="/Users/tim/timdimm_test.esq")
s

{'SequenceQueue': {'@version': '2.5', 'Observer': 'timDIMM', 'GuideDeviation': {'@enabled': 'false', '#text': '2'}, 'GuideStartDeviation': {'@enabled': 'false', '#text': '2'}, 'Autofocus': {'@enabled': 'false', '#text': '0'}, 'RefocusOnTemperatureDelta': {'@enabled': 'false', '#text': '1'}, 'RefocusEveryN': {'@enabled': 'false', '#text': '60'}, 'RefocusOnMeridianFlip': {'@enabled': 'false'}, 'Job': {'Exposure': '0.001', 'Format': 'Mono', 'Encoding': 'FITS', 'Binning': {'X': '1', 'Y': '1'}, 'Frame': {'X': '0', 'Y': '0', 'W': '1608', 'H': '1104'}, 'Temperature': {'@force': 'false', '#text': '0'}, 'Filter': None, 'Type': 'Light', 'Prefix': {'FilterEnabled': '0', 'ExpEnabled': '0', 'TimeStampEnabled': '0'}, 'Count': '1', 'Delay': '0', 'PreCaptureScript': 'timdimm_precapture', 'PostCaptureScript': 'timdimm_postcapture', 'PreJobScript': 'timdimm_prejob', 'PostJobScript': 'timdimm_postjob', 'FITSDirectory': './', 'PlaceholderFormat': '/%t/%T/%F/%t_%T_%F', 'PlaceholderSuffix': '3', 'UploadMode

In [4]:
sched

{'SchedulerList': {'@version': '1.5', 'Profile': 'timdimm_sim', 'Job': [], 'SchedulerAlgorithm': {'@value': '1'}, 'ErrorHandlingStrategy': {'@value': '1', 'delay': '0'}, 'StartupProcedure': {'Procedure': [{'@value': '/home/tim/MMT/timdimm_tng/scripts/ekos/startup.py', '#text': 'StartupScript'}, 'UnparkDome', 'UnparkMount']}, 'ShutdownProcedure': {'Procedure': ['ParkMount', 'ParkDome', {'@value': '/home/tim/MMT/timdimm_tng/scripts/ekos/shutdown.py', '#text': 'ShutdownScript'}]}}}

In [8]:
stars = Table.read("/home/tim/MMT/timdimm_tng/src/timdimm_tng/star_list.ecsv")
stars

HR,Name,RA2000,Dec2000,Vmag,B-V,SED,SpType,Duplicity,Coordinates
,,,,,,,,,"deg,deg"
int64,str14,str8,str9,float64,float64,str3,str12,str15,SkyCoord
98,Bet Hyi,0:25:45,-77:15:15,2.8,0.62,G05,G2IV,--,"6.437793155,-77.25424612"
99,Ankaa,0:26:17,-42:18:22,2.39,1.09,K03,K0III,"0.1"" dm=0.0","6.571047515,-42.305987194"
188,Diphda,0:43:35,-17:59:12,2.04,1.02,K03,G9.5IIICH-1,--,"10.897378736,-17.986606316"
472,Achernar,1:37:43,-57:14:12,0.46,-0.16,B55,B3Vpe,--,"24.428522833,-57.236752806"
591,Alp Hyi,1:58:46,-61:34:11,2.86,0.28,F05,F0V,--,"29.69224409758834,-61.56982215319751"
1790,Bellatrix,5:25:08,+06:20:59,1.64,-0.22,B05,B2III,"179"" dm=10.5","81.282763557,6.349703264"
1865,Arneb,5:32:44,-17:49:20,2.58,0.21,F05,F0Ib,"AB 36"" dm=8.5","83.18256716167,-17.82228927222"
1903,Alnilam,5:36:13,-01:12:07,1.7,-0.19,B05,B0Ia,"180"" dm=8.7","84.053388941,-1.201919136"


In [64]:
sched = Schedule()
for star in stars:
    if star['Vmag'] > 2.5:
        priority = 15
    elif star['Vmag'] < 0.5:
        priority = 5
    else:
        priority = 10
    obs = Observation(
        target=star['Name'],
        ra=star['Coordinates'].ra.value,
        dec=star['Coordinates'].dec.value,
        priority=priority
    )
    sched.add_observation(dict(obs))

In [65]:
sched.to_xml("/home/tim/full_test.esl")

In [28]:
xmltodict.unparse(sched['SchedulerList']['Job'][0])

ValueError: Document must have exactly one root.

In [40]:
dict(sched['SchedulerList']['Job'][0])

{'Name': 'Bet Hyi',
 'Priority': '15',
 'Coordinates': {'J2000RA': '6.437793155', 'J2000DE': '-77.25424612'},
 'PositionAngle': None,
 'Sequence': '/home/tim/MMT/timdimm_tng/src/timdimm_tng/templates/timdimm_sequence.esq',
 'StartupCondition': {'Condition': 'ASAP'},
 'Constraints': {'Constraint': [{'@value': '30', '#text': 'MinimumAltitude'},
   {'@value': '20', '#text': 'MoonSeparation'},
   'EnforceWeather',
   'EnforceTwilight']},
 'CompletionCondition': {'Condition': 'Loop'},
 'Steps': {'Step': ['Track', 'Align']}}

In [42]:
xmltodict.unparse(dict(test['SchedulerList']['Job'][0]))

ValueError: Document must have exactly one root.

In [35]:
with open("/home/tim/MMT/timdimm_tng/src/timdimm_tng/templates/timdimm_schedule_template.esl", 'r') as fp:
    test = xmltodict.parse(fp.read())

In [36]:
test

{'SchedulerList': {'@version': '1.5',
  'Profile': 'timdimm_sim',
  'Job': [{'Name': 'Target',
    'Priority': '10',
    'Coordinates': {'J2000RA': None, 'J2000DE': None},
    'PositionAngle': None,
    'Sequence': '/home/tim/MMT/timdimm_tng/src/timdimm_tng/templates/timdimm_sequence.esq',
    'StartupCondition': {'Condition': 'ASAP'},
    'Constraints': {'Constraint': [{'@value': '30',
       '#text': 'MinimumAltitude'},
      {'@value': '20', '#text': 'MoonSeparation'},
      'EnforceWeather',
      'EnforceTwilight']},
    'CompletionCondition': {'Condition': 'Loop'},
    'Steps': {'Step': ['Track', 'Align']}},
   {'Name': 'Target',
    'Priority': '10',
    'Coordinates': {'J2000RA': None, 'J2000DE': None},
    'PositionAngle': None,
    'Sequence': '/home/tim/MMT/timdimm_tng/src/timdimm_tng/templates/timdimm_sequence.esq',
    'StartupCondition': {'Condition': 'ASAP'},
    'Constraints': {'Constraint': [{'@value': '30',
       '#text': 'MinimumAltitude'},
      {'@value': '20', '#

In [62]:
sched['SchedulerList']['Job'] = sched['SchedulerList']['Job'][0:1]

In [63]:
xmltodict.unparse(sched, pretty=True)

'<?xml version="1.0" encoding="utf-8"?>\n<SchedulerList version="1.5">\n\t<Profile>timdimm_sim</Profile>\n\t<Job>{\'Name\': \'Bet Hyi\', \'Priority\': \'15\', \'Coordinates\': {\'J2000RA\': \'6.437793155\', \'J2000DE\': \'-77.25424612\'}, \'PositionAngle\': None, \'Sequence\': \'/home/tim/MMT/timdimm_tng/src/timdimm_tng/templates/timdimm_sequence.esq\', \'StartupCondition\': {\'Condition\': \'ASAP\'}, \'Constraints\': {\'Constraint\': [{\'@value\': \'30\', \'#text\': \'MinimumAltitude\'}, {\'@value\': \'20\', \'#text\': \'MoonSeparation\'}, \'EnforceWeather\', \'EnforceTwilight\']}, \'CompletionCondition\': {\'Condition\': \'Loop\'}, \'Steps\': {\'Step\': [\'Track\', \'Align\']}}</Job>\n\t<SchedulerAlgorithm value="1"></SchedulerAlgorithm>\n\t<ErrorHandlingStrategy value="1">\n\t\t<delay>0</delay>\n\t</ErrorHandlingStrategy>\n\t<StartupProcedure>\n\t\t<Procedure value="/home/tim/MMT/timdimm_tng/scripts/ekos/startup.py">StartupScript</Procedure>\n\t\t<Procedure>UnparkDome</Procedure>\n\

In [43]:
len(sched['SchedulerList']['Job'])

75

In [60]:
xmltodict.unparse(test, pretty=True)

'<?xml version="1.0" encoding="utf-8"?>\n<SchedulerList version="1.5">\n\t<Profile>timdimm_sim</Profile>\n\t<Job>\n\t\t<Name>Target</Name>\n\t\t<Priority>10</Priority>\n\t\t<Coordinates>\n\t\t\t<J2000RA></J2000RA>\n\t\t\t<J2000DE></J2000DE>\n\t\t</Coordinates>\n\t\t<PositionAngle></PositionAngle>\n\t\t<Sequence>/home/tim/MMT/timdimm_tng/src/timdimm_tng/templates/timdimm_sequence.esq</Sequence>\n\t\t<StartupCondition>\n\t\t\t<Condition>ASAP</Condition>\n\t\t</StartupCondition>\n\t\t<Constraints>\n\t\t\t<Constraint value="30">MinimumAltitude</Constraint>\n\t\t\t<Constraint value="20">MoonSeparation</Constraint>\n\t\t\t<Constraint>EnforceWeather</Constraint>\n\t\t\t<Constraint>EnforceTwilight</Constraint>\n\t\t</Constraints>\n\t\t<CompletionCondition>\n\t\t\t<Condition>Loop</Condition>\n\t\t</CompletionCondition>\n\t\t<Steps>\n\t\t\t<Step>Track</Step>\n\t\t\t<Step>Align</Step>\n\t\t</Steps>\n\t</Job>\n\t<Job>\n\t\t<Name>Target</Name>\n\t\t<Priority>10</Priority>\n\t\t<Coordinates>\n\t\t\t

In [59]:
sched

{'SchedulerList': {'@version': '1.5', 'Profile': 'timdimm_sim', 'Job': [{'Name': 'Bet Hyi', 'Priority': '15', 'Coordinates': {'J2000RA': '6.437793155', 'J2000DE': '-77.25424612'}, 'PositionAngle': None, 'Sequence': '/home/tim/MMT/timdimm_tng/src/timdimm_tng/templates/timdimm_sequence.esq', 'StartupCondition': {'Condition': 'ASAP'}, 'Constraints': {'Constraint': [{'@value': '30', '#text': 'MinimumAltitude'}, {'@value': '20', '#text': 'MoonSeparation'}, 'EnforceWeather', 'EnforceTwilight']}, 'CompletionCondition': {'Condition': 'Loop'}, 'Steps': {'Step': ['Track', 'Align']}}, {'Name': 'Ankaa', 'Priority': '10', 'Coordinates': {'J2000RA': '6.571047515', 'J2000DE': '-42.305987194'}, 'PositionAngle': None, 'Sequence': '/home/tim/MMT/timdimm_tng/src/timdimm_tng/templates/timdimm_sequence.esq', 'StartupCondition': {'Condition': 'ASAP'}, 'Constraints': {'Constraint': [{'@value': '30', '#text': 'MinimumAltitude'}, {'@value': '20', '#text': 'MoonSeparation'}, 'EnforceWeather', 'EnforceTwilight']